In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from xgboost import XGBClassifier

# import time
# # tmp sleep
# time.sleep(3600)
# print('sleep done')

# load feats
train_x,test_x = [],[]
for feat in glob.glob('../features/*.pkl'):
#     if 'tfidf' in feat or 'lr' in feat or 'mnb' in feat:
#         continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/lr_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/mnb_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/gru_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/other_feat.pkl
(95851, 16) (226998, 16)
file path ../features/lstm_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/mnb_feat2.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/simple_rnn_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/tfidf_feat1.pkl
(95851, 30) (226998, 30)
file path ../features/lstm_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lr_feat2.pkl
(95851, 6) (226998, 6)
file path ../features/tfidf_feat2.pkl
(9585

In [2]:
print(train_y[:10])
print(len(train_y))
print(sum(train_y[:,0]))

[[1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
95851
9237


In [3]:
from sklearn.model_selection import KFold
def simple_ens(model_name,k=3):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    test_pred = np.zeros((226998,6))
    all_train_loss_l,all_val_loss_l = 0,0
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        
        # train for each class
        for i in range(6):
            if model_name == 'lr':
                model = LogisticRegression()
            if model_name == 'xgb':
                model = XGBClassifier()
            model.fit(curr_x,curr_y[:,i])
            print(i)
            try:
                curr_train_loss = log_loss(curr_y[:,i],model.predict_proba(curr_x))
                curr_val_loss = log_loss(hold_out_y[:,i],model.predict_proba(hold_out_x))
                print(curr_train_loss,curr_val_loss)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
            except:
                pass
            curr_test_pred = model.predict_proba(test_x)[:,1]
            test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred

print('done')
            
                
            

done


In [4]:
xgb_res = simple_ens('xgb',k=3)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_all_ens_csv.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

0
0.0817318215346 0.0859880002129
1
0.017718525232 0.0224443283163
2
0.0429840342473 0.0461487043987
3
0.00459540953861 0.00904503380752
4
0.0540994420691 0.062501480019
5
0.0155795293439 0.0195527339888
this fold avg train 0.0361181269943 avg val 0.0409467134572
0
0.080396795704 0.0895978380962
1
0.017882252786 0.0220953791331
2
0.0419689856318 0.0487188099849
3
0.00497928105057 0.00881785479514
4
0.0552432054081 0.0596306472198
5
0.0155108976935 0.0208331598207
this fold avg train 0.0359969030457 avg val 0.0416156148416
0
0.0810244875992 0.0885507809827
1
0.0174504480731 0.0228472442639
2
0.0413791103358 0.0501549052156
3
0.00519341388668 0.00772611344052
4
0.0550069822037 0.060284373122
5
0.0156126730418 0.0204347381996
this fold avg train 0.0359445191901 avg val 0.041666359204
all train avg 0.0360198497433 all val avg 0.041409562501
         id     toxic  severe_toxic   obscene    threat    insult  \
0   6044863  0.001939      0.000080  0.000567  0.000057  0.000385   
1   6102620  